<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型
假設每個輸入項xi不再只是單一數值，而是一個由xil,...,xik所組成的向量k。  
多元遞歸模型架設如下：


```
yi = α+β₁χɪʟ+...+βkχɪʟ
```
在多元迴歸模型中，參數向量通常以β來表示。  
若把常數項也包含進來，只需多加一個欄位。
* beta = [alpha, beta_1 , ... ,beta_k]
* x_i = [1, x_i1 , ... ,x_ik]
  




於是我們的模型就變成：  
[常數項，朋友數量，每天工作時數，有博士學位與否]

In [ ]:
from typing import List

Vector = List[float]

def dot(v:Vector,w:Vector)->float:
  #計算v_1*w_1+... +v_n*w_n
  assert len(v)==len(w),"兩個向量必須有相同的維度"

  return sum(v_i*w_i for v_i,w_i in zip(v,w))

In [ ]:
def predict (x:Vector,beta:Vector) -> float:
  """假設每個x_i的第一個元素都是1"""
  return dot(x,beta)

# 關於最小平方模型的進一步假設

1. x的每個元素必須是**線性獨立**的：每個元素皆無法透過其他元素加總來得到。  
若該假設不成立，則無法得估計出β。
2. x中的每個元素全都與誤差項ε無關。  
若該假設不成立，估計β時會發生系統性錯誤。

# 套入模型
目標：找到某組beta係數值，讓誤差平方得到最小化的結果

## 誤差函數

In [ ]:
from typing import List

def error(x:Vector,y:float,beta:Vector) -> float:
  return predict(x,beta) - y

def squared_error(x:Vector,y:float,beta:Vector) -> float:
  return error(x,y,beta)**2

進行測試

In [ ]:
x = [1,2,3]
y = 30
beta = [4,4,4] #預測值為 4+8+12 = 24

assert error(x,y,beta) == -6
assert squared_error(x,y,beta) == 36

## 計算梯度
透過微積分

In [ ]:
def squerror_gradient(x:Vector,y:float,beta:Vector) -> float:
  err = error(x,y,beta)
  return [2*err*x_i for x_i in x]

In [ ]:
assert squerror_gradient(x,y,beta) == [-12,-24,-36]

## 1. 梯度遞減 (書本上作法)
用來找出最佳的beta值

In [ ]:
import random
import tqdm

from typing import List

Vector = List[float]

def add( v:Vector, w:Vector) -> Vector:
  assert len(v) == len(w) ,"兩個向量必須有相同的維度"

  return [ v_i+w_i for v_i,w_i in zip(v,w)]

def vector_sum(vectors:List[Vector]) -> Vector:
  #先檢查vertors這個向量列表是否為空
  assert vectors,"列表中沒有向量!"

  #檢查vertors 向量列表內的所有向量都具有相同的維度
  num_elements=len(vectors[0])
  assert all(len(v)==num_elements for v in vectors),"向量維度不一致"

  #所有vectors[i]相加起來，是結果的第i個元素值
  return [sum(vector[i] for vector in vectors) for i in range(num_elements)]


def scalar_multiply(c:float,v:Vector) -> Vector:
  return [c*v_i for v_i in v]

def vector_mean(vectors:List[Vector])->Vector:
  n=len(vectors)
  return scalar_multiply(1/n,vector_sum(vectors))

def gradient_step(v:Vector,gradient:Vector,step_size:float) -> Vector:
  """從v沿著gradient的方向移動step_size的距離"""
  assert len(v) == len(gradient)
  step = scalar_multiply(step_size,gradient)
  return add(v,step)

In [ ]:
def least_squares_fit(
          xs:List[Vector],
          ys:List[float],
          learning_rate:float=0.001,
          num_steps:int=1000,
          batch_size:int=1)->Vector:
  """
  找出最小化平方誤差和的beta值
  假設模型y = dot(x,beta)
  """

  # 一開始先使隨機方式做出猜測
  guess = [random.random() for _ in xs[0]]

  for _ in tqdm.trange(num_steps,desc="least squares fit"):
    for start in range(0,len(xs),batch_size):
      batch_xs = xs[start:start+batch_size]
      batch_ys = ys[start:start+batch_size]

      gradient = vector_mean([squerror_gradient(x,y,guess) for x,y in zip(batch_xs,batch_ys)])

      guess = gradient_step(guess,gradient,-learning_rate)
  
  return guess

## 套用到資料中

In [46]:
inputs = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

In [48]:
import numpy as np

random.seed(0)

# 嘗試使用錯誤的方式選擇num_iters和step_size
# 這會需要花一點時間
learning_rate = 0.001

beta = least_squares_fit(inputs,daily_minutes_good,learning_rate ,5000,25)

assert 30.50 < beta[0] < 30.70 #常數
assert 0.96 < beta[1] < 1.00 #朋友的數量
assert -1.89 < beta[2] < -1.85 #每天的工作時數
assert 0.91 < beta[3] < 0.94 #有博士學問的話

np.round(np.array(beta),3)

least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 1707.40it/s]


array([30.515,  0.975, -1.851,  0.914])

在實務上不會使用梯度遞減的方式來計算線性回歸的方法，  

## ✪ [線性代數投影求解](https://ithelp.ithome.com.tw/articles/10186400) (網路資源)
![公式解](http://i.imgur.com/EngTecm.png)  
利用numpy函式庫來協助運算


In [41]:
import numpy as np
x = np.array(inputs)
y = np.array(daily_minutes_good)

# A = (X^T*X)⁻¹
transpose_x = np.transpose(x)
temp = transpose_x @ x
A = np.linalg.inv(temp)

# D = X^T * y
D = transpose_x @ y

# β = A*D = 最終解 = (X^T*X)⁻¹ * X^T * y
β = A @ D
np.round(β,3) #將最終的解,取小數點後三位

array([30.579,  0.973, -1.865,  0.923])

## 運算後，得到的方程式是：
```
minutes = 30.579 + 0.973 friends - 1.865 woek hours + 0.923 phd
```


# 解釋模型
　　
**所有其他因素皆相等的情況下**：
* 每額外增加一個朋友，每天花在網站上的時間就會增加將近1分鐘。
* 使用者的工作時間美額外增加１小時，每天花在網站上的時間就會減少將近２分鐘。
* 每個擁有博士學位的使用者，每天都會在網站上多待近1分鐘。
  
這些變數彼此間的交互關係，對於朋友多或少的人來說工作時數的影響**可能是不相同**的。  

# 套入優度
利用**R平方**的值來觀察：

In [49]:
def mean(xs:List[float]) -> float:
  return sum(xs) / len(xs)

def de_mean(xs:List[float]) -> List[float]:
  x_bar = mean(xs)
  return [x - x_bar for x in xs] 


In [58]:
def error(x , y , beta):
    return y - predict(x,beta)

def sum_of_sqerrors(x , y , beta):
    return sum( error(x_i , y_i , beta) ** 2
               for x_i, y_i in zip(x, y))

def total_sum_squares(y:Vector) ->float:
  """每個y_i與平均值之間的差值的總平方和，即「總變異量」"""
  return sum(v**2 for v in de_mean(y))

def mulity_r_squares(xs:List[float],ys:Vector,beta:Vector) ->float:
  """
  模型掌握到y變異量的比率，即(1-模型未掌握到y變異量的比率)
  """
  sum_of_squared_errors = sum(error(x,y,beta)**2 for x , y in zip(xs,ys))
  return 1.0 - (sum_of_squared_errors/total_sum_squares(ys))

def total_sum_of_squares(y):
    """the total squared variation of y_i's from their mean"""
    return sum(v ** 2 for v in de_mean(y))

In [59]:
rsq= mulity_r_squares(inputs,daily_minutes_good,beta)
assert 0.67 < rsq < 0.68
print("%.3f"%rsq)

0.680


在多元迴歸模型中，還要觀察各個係數的標準差，以衡量所得到每個βi估計值。  
  
**衡量誤差的假設**：
誤差項εi是一個平均值為0，標準差σ的**獨立常態隨機變數**。  
模型中係數的標準差越大，該係數的可靠程度就越低。  

## Bootstrap (重複取樣)
假設我們有一組其中包含N個資料的樣本，是根據某種未知的分布所生成的：

```
data = get_sample(num_points = n
```
如果重複取得多組的樣本組，我們可以計算出許多樣本組的中位數，進而觀察這些中位數的分布情況。  
這種情況下，可以「重複採樣(bootstrap)」從原本的資料集內，


# 迴歸係數的標準差

# 正則化